<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
diff --git a/build.sh b/build.sh
index 3cc0490..8d74558 100755
--- a/build.sh
+++ b/build.sh
@@ -492,25 +492,25 @@ write_commit_file() {
 build_lite_x86_64_jni_and_jar()
 {
     # copy x86 so
-    local inference_or_train=inference
     local is_train=off
     cd ${BASEPATH}/output/tmp
-    if [ -f "mindspore-lite-${VERSION_STR}-train-linux-x64.tar.gz" ]; then
-        inference_or_train=train
-        is_train=on
+    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
+      echo "is train on"
+      is_train=on
     fi
-    local pkg_name=mindspore-lite-${VERSION_STR}-${inference_or_train}-linux-x64
+    local pkg_name=mindspore-lite-${VERSION_STR}-linux-x64
 
     cd ${BASEPATH}/output/tmp/
     rm -rf ${pkg_name}
     tar -zxf ${BASEPATH}/output/tmp/${pkg_name}.tar.gz
     rm -rf ${LITE_JAVA_PATH}/java/linux_x86/libs/   && mkdir -pv ${LITE_JAVA_PATH}/java/linux_x86/libs/
     rm -rf ${LITE_JAVA_PATH}/native/libs/linux_x86/ && mkdir -pv ${LITE_JAVA_PATH}/native/libs/linux_x86/
-    cp ./${pkg_name}/inference/lib/*.so* ${LITE_JAVA_PATH}/java/linux_x86/libs/
-    cp ./${pkg_name}/inference/lib/*.so* ${LITE_JAVA_PATH}/native/libs/linux_x86/
-    if [ -f "mindspore-lite-${VERSION_STR}-train-linux-x64.tar.gz" ]; then
-        cp ./${pkg_name}/inference/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/linux_x86/libs/
-        cp ./${pkg_name}/inference/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/linux_x86/
+    cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/java/linux_x86/libs/
+    cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/native/libs/linux_x86/
+    if ["${MSLITE_ENABLE_TRAIN}"]; then
+        echo "cp libjpeg"
+        cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/linux_x86/libs/
+        cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/linux_x86/
     fi
     # build jni so
     cd ${BASEPATH}/mindspore/lite/build
@@ -525,7 +525,7 @@ build_lite_x86_64_jni_and_jar()
     fi
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/java/linux_x86/libs/
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/native/libs/linux_x86/
-    cp ./libmindspore-lite-jni.so ${BASEPATH}/output/tmp/${pkg_name}/inference/lib/
+    cp ./libmindspore-lite-jni.so ${BASEPATH}/output/tmp/${pkg_name}/runtime/lib/
 
     # build java common
     cd ${LITE_JAVA_PATH}/java/common
@@ -537,7 +537,7 @@ build_lite_x86_64_jni_and_jar()
     cd ${LITE_JAVA_PATH}/java/linux_x86/
     gradle clean
     gradle releaseJar
-    cp ./build/lib/jar/*.jar ${BASEPATH}/output/tmp/${pkg_name}/inference/lib/
+    cp ./build/lib/jar/*.jar ${BASEPATH}/output/tmp/${pkg_name}/runtime/lib/
 
     # package
     cd ${BASEPATH}/output/tmp
@@ -643,24 +643,23 @@ build_lite_arm64_and_jni() {
     # build arm64
     build_lite "arm64"
     # copy arm64 so
-    local inference_or_train=inference
     local is_train=off
-    if [ -f "${BASEPATH}/output/mindspore-lite-${VERSION_STR}-train-android-aarch64.tar.gz" ]; then
-        inference_or_train=train
-        is_train=on
+    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
+      echo "is train on"
+      is_train=on
     fi
-    local pkg_name=mindspore-lite-${VERSION_STR}-${inference_or_train}-android-aarch64
+    local pkg_name=mindspore-lite-${VERSION_STR}-android-aarch64
     cd "${BASEPATH}/mindspore/lite/build"
 
     rm -rf ${pkg_name}
     tar -zxf ${BASEPATH}/output/${pkg_name}.tar.gz
     rm -rf ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/ && mkdir -p ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
     rm -rf ${LITE_JAVA_PATH}/native/libs/arm64-v8a/   && mkdir -p ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
-    cp ./${pkg_name}/${inference_or_train}/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
-    cp ./${pkg_name}/${inference_or_train}/lib/*.so* ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
-    if [ -f "${BASEPATH}/output/mindspore-lite-${VERSION_STR}-train-android-aarch64.tar.gz" ]; then
-      cp ./${pkg_name}/train/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
-      cp ./${pkg_name}/train/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
+    cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
+    cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
+    if ["${MSLITE_ENABLE_TRAIN}"]; then
+      cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
+      cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
     fi
     # build jni so
     [ -n "${BASEPATH}" ] && rm -rf java/jni && mkdir -pv java/jni
@@ -683,24 +682,23 @@ build_lite_arm32_and_jni() {
     # build arm32
     build_lite "arm32"
     # copy arm32 so
-    local inference_or_train=inference
     local is_train=off
-    if [ -f "${BASEPATH}/output/mindspore-lite-${VERSION_STR}-train-android-aarch32.tar.gz" ]; then
-        inference_or_train=train
-        is_train=on
+    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
+      echo "is train on"
+      is_train=on
     fi
-    local pkg_name=mindspore-lite-${VERSION_STR}-${inference_or_train}-android-aarch32
+    local pkg_name=mindspore-lite-${VERSION_STR}-android-aarch32
     cd "${BASEPATH}/mindspore/lite/build"
 
     rm -rf ${pkg_name}
     tar -zxf ${BASEPATH}/output/${pkg_name}.tar.gz
     rm -rf ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/ && mkdir -pv ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
     rm -rf ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/   && mkdir -pv ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
-    cp ./${pkg_name}/${inference_or_train}/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
-    cp ./${pkg_name}/${inference_or_train}/lib/*.so* ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
-    if [ -f "${BASEPATH}/output/mindspore-lite-${VERSION_STR}-train-android-aarch32.tar.gz" ]; then
-      cp ./${pkg_name}/train/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
-      cp ./${pkg_name}/train/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
+    cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
+    cp ./${pkg_name}/runtime/lib/*.so* ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
+    if [ "${MSLITE_ENABLE_TRAIN}" ]; then
+      cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
+      cp ./${pkg_name}/runtime/third_party/libjpeg-turbo/lib/*.so* ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
     fi
 
     # build jni so
diff --git a/cmake/package_lite.cmake b/cmake/package_lite.cmake
index 43ae08f..6b5ab72 100644
--- a/cmake/package_lite.cmake
+++ b/cmake/package_lite.cmake
@@ -8,12 +8,12 @@ set(OBFUSCATOR_ROOT_DIR ${RUNTIME_PKG_NAME}/tools/obfuscator)
 set(CROPPER_ROOT_DIR ${RUNTIME_PKG_NAME}/tools/cropper)
 set(TEST_CASE_DIR ${TOP_DIR}/mindspore/lite/test/build)
 
-set(RUNTIME_DIR ${RUNTIME_PKG_NAME}/inference)
-set(RUNTIME_INC_DIR ${RUNTIME_PKG_NAME}/inference/include)
-set(RUNTIME_LIB_DIR ${RUNTIME_PKG_NAME}/inference/lib)
-set(MIND_DATA_INC_DIR ${RUNTIME_PKG_NAME}/inference/include/dataset)
-set(TURBO_DIR ${RUNTIME_PKG_NAME}/inference/third_party/libjpeg-turbo)
-set(SECUREC_DIR ${RUNTIME_PKG_NAME}/inference/third_party/securec)
+set(RUNTIME_DIR ${RUNTIME_PKG_NAME}/runtime)
+set(RUNTIME_INC_DIR ${RUNTIME_PKG_NAME}/runtime/include)
+set(RUNTIME_LIB_DIR ${RUNTIME_PKG_NAME}/runtime/lib)
+set(MIND_DATA_INC_DIR ${RUNTIME_PKG_NAME}/runtime/include/dataset)
+set(TURBO_DIR ${RUNTIME_PKG_NAME}/runtime/third_party/libjpeg-turbo)
+set(SECUREC_DIR ${RUNTIME_PKG_NAME}/runtime/third_party/securec)
 set(MINDSPORE_LITE_LIB_NAME libmindspore-lite)
 set(BENCHMARK_NAME benchmark)
 set(BENCHMARK_ROOT_DIR ${RUNTIME_PKG_NAME}/tools/benchmark)
diff --git a/mindspore/lite/CMakeLists.txt b/mindspore/lite/CMakeLists.txt
index 8185ee6..fb5011f 100644
--- a/mindspore/lite/CMakeLists.txt
+++ b/mindspore/lite/CMakeLists.txt
@@ -129,26 +129,24 @@ add_compile_options(-fPIC)
 
 if(SUPPORT_TRAIN)
     set(BUILD_MINDDATA "full")
-    set(TRAIN_OR_INFERENCE "train")
 else()
-    set(TRAIN_OR_INFERENCE "inference")
 endif()
 
 if(PLATFORM_ARM64)
-    set(RUNTIME_COMPONENT_NAME ${TRAIN_OR_INFERENCE}-android-aarch64)
+    set(RUNTIME_COMPONENT_NAME "android-aarch64")
 elseif(PLATFORM_ARM32)
-    set(RUNTIME_COMPONENT_NAME ${TRAIN_OR_INFERENCE}-android-aarch32)
+    set(RUNTIME_COMPONENT_NAME "android-aarch32")
 elseif(WIN32)
     execute_process(COMMAND "${CMAKE_C_COMPILER}" -dumpmachine
         OUTPUT_VARIABLE i686_or_x86_64
     )
     if(i686_or_x86_64 MATCHES "^i686-")
-        set(RUNTIME_COMPONENT_NAME ${TRAIN_OR_INFERENCE}-win-x86)
+        set(RUNTIME_COMPONENT_NAME "win-x86")
     else()
-        set(RUNTIME_COMPONENT_NAME ${TRAIN_OR_INFERENCE}-win-x64)
+        set(RUNTIME_COMPONENT_NAME "win-x64")
     endif()
 else()
-    set(RUNTIME_COMPONENT_NAME ${TRAIN_OR_INFERENCE}-linux-x64)
+    set(RUNTIME_COMPONENT_NAME "linux-x64")
 endif()
 
 string(REPLACE "/mindspore/lite" "" TOP_DIR ${CMAKE_CURRENT_SOURCE_DIR})
diff --git a/mindspore/lite/minddata/example/CMakeLists.txt b/mindspore/lite/minddata/example/CMakeLists.txt
index a5ac3ac..c6c753f 100644
--- a/mindspore/lite/minddata/example/CMakeLists.txt
+++ b/mindspore/lite/minddata/example/CMakeLists.txt
@@ -4,7 +4,7 @@ set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -Werror -Wall -fPIC -std=c++17")
 
 set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -Wno-sign-compare")
 
-set(MS_DIR "${CMAKE_CURRENT_SOURCE_DIR}/mindspore-lite-1.2.0-inference-linux-x64/inference")
+set(MS_DIR "${CMAKE_CURRENT_SOURCE_DIR}/mindspore-lite-1.2.0-linux-x64/runtime")
 
 include_directories(${MS_DIR})
 
diff --git a/mindspore/lite/test/st/run_net_train.sh b/mindspore/lite/test/st/run_net_train.sh
index 7eed84f..beb366a 100755
--- a/mindspore/lite/test/st/run_net_train.sh
+++ b/mindspore/lite/test/st/run_net_train.sh
@@ -35,8 +35,8 @@ function Run_Export(){
 # Run converter on x86 platform:
 function Run_Converter() {
     cd ${x86_path} || exit 1
-    tar -zxf mindspore-lite-${version}-train-linux-x64.tar.gz || exit 1
-    cd ${x86_path}/mindspore-lite-${version}-train-linux-x64/ || exit 1
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    cd ${x86_path}/mindspore-lite-${version}-linux-x64/ || exit 1
 
     cp tools/converter/converter/converter_lite ./ || exit 1
     export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:./tools/converter/lib/:./tools/converter/third_party/glog/lib
@@ -73,8 +73,8 @@ function Run_Converter() {
 
 # Run on x86 platform:
 function Run_x86() {
-    cd ${x86_path}/mindspore-lite-${version}-train-linux-x64 || return 1
-    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:./inference/lib:./inference/third_party/libjpeg-turbo/lib
+    cd ${x86_path}/mindspore-lite-${version}-linux-x64 || return 1
+    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:./runtime/lib:./runtime/third_party/libjpeg-turbo/lib
     # Run mindspore converted train models:
     fail=0
     while read line; do
@@ -137,24 +137,24 @@ function Run_arm() {
 
     # Unzip
     cd ${arm_path} || exit 1
-    tar -zxf mindspore-lite-${version_arm}-train-android-${process_unit}.tar.gz || exit 1
+    tar -zxf mindspore-lite-${version_arm}-android-${process_unit}.tar.gz || exit 1
 
     # If build with minddata, copy the minddata related libs
     cd ${benchmark_train_test_path} || exit 1
-    if [ -f ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/lib/libminddata-lite.so ]; then
-        cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/third_party/libjpeg-turbo/lib/libjpeg.so* ${benchmark_train_test_path}/ || exit 1
-        cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/third_party/libjpeg-turbo/lib/libturbojpeg.so* ${benchmark_train_test_path}/ || exit 1
-        cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/lib/libminddata-lite.so ${benchmark_train_test_path}/libminddata-lite.so || exit 1
+    if [ -f ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/lib/libminddata-lite.so ]; then
+        cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/third_party/libjpeg-turbo/lib/libjpeg.so* ${benchmark_train_test_path}/ || exit 1
+        cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/third_party/libjpeg-turbo/lib/libturbojpeg.so* ${benchmark_train_test_path}/ || exit 1
+        cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/lib/libminddata-lite.so ${benchmark_train_test_path}/libminddata-lite.so || exit 1
     fi
     if [ "$1" == arm64 ] || [ "$1" == arm32 ]; then
-        cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/third_party/hiai_ddk/lib/libhiai.so ${benchmark_train_test_path}/libhiai.so || exit 1
-        cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_train_test_path}/libhiai_ir.so || exit 1
-        cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_train_test_path}/libhiai_ir_build.so || exit 1
+        cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_train_test_path}/libhiai.so || exit 1
+        cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_train_test_path}/libhiai_ir.so || exit 1
+        cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_train_test_path}/libhiai_ir_build.so || exit 1
     fi
 
-    cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/lib/libmindspore-lite.so ${benchmark_train_test_path}/libmindspore-lite.so || exit 1
-    cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/inference/lib/libmindspore-lite-train.so ${benchmark_train_test_path}/libmindspore-lite-train.so || exit 1
-    cp -a ${arm_path}/mindspore-lite-${version_arm}-train-android-${process_unit}/tools/benchmark_train/benchmark_train ${benchmark_train_test_path}/benchmark_train || exit 1
+    cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/lib/libmindspore-lite.so ${benchmark_train_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/runtime/lib/libmindspore-lite-train.so ${benchmark_train_test_path}/libmindspore-lite-train.so || exit 1
+    cp -a ${arm_path}/mindspore-lite-${version_arm}-android-${process_unit}/tools/benchmark_train/benchmark_train ${benchmark_train_test_path}/benchmark_train || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_train_test_path} /data/local/tmp/ > ${adb_push_log_file}
@@ -316,19 +316,19 @@ fi
 echo $train_io_path
 
 arm64_path=${release_path}/android_aarch64
-file=$(ls ${arm64_path}/*train-android-aarch64.tar.gz)
+file=$(ls ${arm64_path}/*android-aarch64.tar.gz)
 file_name="${file##*/}"
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version_arm64=${file_name_array[2]}
 
 arm32_path=${release_path}/android_aarch32
-file=$(ls ${arm32_path}/*train-android-aarch32.tar.gz)
+file=$(ls ${arm32_path}/*android-aarch32.tar.gz)
 file_name="${file##*/}"
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version_arm32=${file_name_array[2]}
 
 x86_path=${release_path}/ubuntu_x86
-file=$(ls ${x86_path}/*train-linux-x64.tar.gz)
+file=$(ls ${x86_path}/*linux-x64.tar.gz)
 file_name="${file##*/}"
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version=${file_name_array[2]}
diff --git a/mindspore/lite/test/st/scripts/run_benchmark_arm.sh b/mindspore/lite/test/st/scripts/run_benchmark_arm.sh
index fab0a6f..c1cc3eb 100644
--- a/mindspore/lite/test/st/scripts/run_benchmark_arm.sh
+++ b/mindspore/lite/test/st/scripts/run_benchmark_arm.sh
@@ -4,8 +4,8 @@
 function Run_Converter() {
     # Unzip x86 runtime and converter
     cd ${x86_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-linux-x64.tar.gz || exit 1
-    cd ${x86_path}/mindspore-lite-${version}-inference-linux-x64/ || exit 1
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    cd ${x86_path}/mindspore-lite-${version}-linux-x64/ || exit 1
 
     cp tools/converter/converter/converter_lite ./ || exit 1
     export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:./tools/converter/lib/:./tools/converter/third_party/glog/lib
@@ -424,9 +424,9 @@ function Run_Converter() {
 function Run_arm64_codegen() {
     echo "ANDROID_NDK: ${ANDROID_NDK}" >> ${run_arm64_fp32_codegen_log_file}
     cd ${arm64_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch64.tar.gz || exit 1
-    local PKG_PATH=${arm64_path}/mindspore-lite-${version}-inference-android-aarch64
-    local CODEGEN_PATH=${x86_path}/mindspore-lite-${version}-inference-linux-x64/tools/codegen
+    tar -zxf mindspore-lite-${version}-android-aarch64.tar.gz || exit 1
+    local PKG_PATH=${arm64_path}/mindspore-lite-${version}-android-aarch64
+    local CODEGEN_PATH=${x86_path}/mindspore-lite-${version}-linux-x64/tools/codegen
 
     rm -rf ${build_path}
     mkdir -p ${build_path}
@@ -512,9 +512,9 @@ function Run_arm64_codegen() {
 function Run_arm32_codegen() {
     echo "ANDROID_NDK: ${ANDROID_NDK}" >> ${run_arm32_fp32_codegen_log_file}
     cd ${arm32_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch32.tar.gz || exit 1
-    local PKG_PATH=${arm32_path}/mindspore-lite-${version}-inference-android-aarch32
-    local CODEGEN_PATH=${x86_path}/mindspore-lite-${version}-inference-linux-x64/tools/codegen
+    tar -zxf mindspore-lite-${version}-android-aarch32.tar.gz || exit 1
+    local PKG_PATH=${arm32_path}/mindspore-lite-${version}-android-aarch32
+    local CODEGEN_PATH=${x86_path}/mindspore-lite-${version}-linux-x64/tools/codegen
 
     rm -rf ${build_path}
     mkdir -p ${build_path}
@@ -600,19 +600,19 @@ function Run_arm32_codegen() {
 # Run on arm64 platform:
 function Run_arm64() {
     cd ${arm64_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch64.tar.gz || exit 1
+    tar -zxf mindspore-lite-${version}-android-aarch64.tar.gz || exit 1
 
     # If build with minddata, copy the minddata related libs
     cd ${benchmark_test_path} || exit 1
-    if [ -f ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libminddata-lite.so ]; then
-        cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+    if [ -f ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ]; then
+        cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
     fi
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
 
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -1021,18 +1021,18 @@ function Run_arm64() {
 # Run on arm32 platform:
 function Run_arm32() {
     cd ${arm32_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch32.tar.gz || exit 1
+    tar -zxf mindspore-lite-${version}-android-aarch32.tar.gz || exit 1
 
     # If build with minddata, copy the minddata related libs
     cd ${benchmark_test_path} || exit 1
-    if [ -f ${arm32_path}/mindspore-lite-${version}-inference-android-aarch32/inference/lib/libminddata-lite.so ]; then
-        cp -a ${arm32_path}/mindspore-lite-${version}-inference-android-aarch32/inference/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+    if [ -f ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libminddata-lite.so ]; then
+        cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
     fi
-    cp -a ${arm32_path}/mindspore-lite-${version}-inference-android-aarch32/inference/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
-    cp -a ${arm32_path}/mindspore-lite-${version}-inference-android-aarch32/inference/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
-    cp -a ${arm32_path}/mindspore-lite-${version}-inference-android-aarch32/inference/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
-    cp -a ${arm32_path}/mindspore-lite-${version}-inference-android-aarch32/inference/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
-    cp -a ${arm32_path}/mindspore-lite-${version}-inference-android-aarch32/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm32_path}/mindspore-lite-${version}-android-aarch32/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -1077,19 +1077,19 @@ function Run_arm32() {
 # Run on arm64-fp16 platform:
 function Run_arm64_fp16() {
     cd ${arm64_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch64.tar.gz || exit 1
+    tar -zxf mindspore-lite-${version}-android-aarch64.tar.gz || exit 1
 
     # If build with minddata, copy the minddata related libs
     cd ${benchmark_test_path} || exit 1
-    if [ -f ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libminddata-lite.so ]; then
-        cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+    if [ -f ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ]; then
+        cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
     fi
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
 
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -1249,18 +1249,18 @@ function Run_arm64_fp16() {
 # Run on armv8.2-a32-fp16 platform:
 function Run_armv82_a32_fp16() {
     cd ${armv82_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch32.tar.gz || exit 1
+    tar -zxf mindspore-lite-${version}-android-aarch32.tar.gz || exit 1
 
     # If build with minddata, copy the minddata related libs
     cd ${benchmark_test_path} || exit 1
-    if [ -f ${armv82_path}/mindspore-lite-${version}-inference-android-aarch32/inference/minddata/lib/libminddata-lite.so ]; then
-        cp -a ${armv82_path}/mindspore-lite-${version}-inference-android-aarch32/inference/minddata/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+    if [ -f ${armv82_path}/mindspore-lite-${version}-android-aarch32/runtime/minddata/lib/libminddata-lite.so ]; then
+        cp -a ${armv82_path}/mindspore-lite-${version}-android-aarch32/runtime/minddata/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
     fi
-    cp -a ${armv82_path}/mindspore-lite-${version}-inference-android-aarch32/inference/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
-    cp -a ${armv82_path}/mindspore-lite-${version}-inference-android-aarch32/inference/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
-    cp -a ${armv82_path}/mindspore-lite-${version}-inference-android-aarch32/inference/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
-    cp -a ${armv82_path}/mindspore-lite-${version}-inference-android-aarch32/inference/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
-    cp -a ${armv82_path}/mindspore-lite-${version}-inference-android-aarch32/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${armv82_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
+    cp -a ${armv82_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
+    cp -a ${armv82_path}/mindspore-lite-${version}-android-aarch32/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
+    cp -a ${armv82_path}/mindspore-lite-${version}-android-aarch32/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${armv82_path}/mindspore-lite-${version}-android-aarch32/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -1485,7 +1485,7 @@ done
 # mkdir train
 
 x86_path=${release_path}/ubuntu_x86
-file_name=$(ls ${x86_path}/*inference-linux-x64.tar.gz)
+file_name=$(ls ${x86_path}/*linux-x64.tar.gz)
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version=${file_name_array[2]}
 
@@ -1582,7 +1582,7 @@ isFailed=0
 if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_codegen" ]]; then
     # Run on arm64
     arm64_path=${release_path}/android_aarch64
-    file_name=$(ls ${arm64_path}/*inference-android-aarch64.tar.gz)
+    file_name=$(ls ${arm64_path}/*android-aarch64.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
@@ -1595,7 +1595,7 @@ fi
 if [[ $backend == "all" || $backend == "arm32_cpu" || $backend == "arm32_codegen" ]]; then
     # Run on arm32
     arm32_path=${release_path}/android_aarch32
-    file_name=$(ls ${arm32_path}/*inference-android-aarch32.tar.gz)
+    file_name=$(ls ${arm32_path}/*android-aarch32.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
@@ -1608,7 +1608,7 @@ fi
 if [[ $backend == "all" || $backend == "arm32_cpu" || $backend == "arm32_fp16" ]]; then
     # Run on armv82-a32-fp16
     armv82_path=${release_path}/android_aarch32
-    file_name=$(ls ${armv82_path}/*inference-android-aarch32.tar.gz)
+    file_name=$(ls ${armv82_path}/*android-aarch32.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
@@ -1622,7 +1622,7 @@ if [[ $backend == "all" || $backend == "arm32_cpu" || $backend == "arm32_fp32" ]
     # Run on arm32
     arm32_path=${release_path}/android_aarch32
     # mv ${arm32_path}/*train-android-aarch32* ./train
-    file_name=$(ls ${arm32_path}/*inference-android-aarch32.tar.gz)
+    file_name=$(ls ${arm32_path}/*android-aarch32.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
@@ -1636,7 +1636,7 @@ if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_fp32" ]
     # Run on arm64
     arm64_path=${release_path}/android_aarch64
     # mv ${arm64_path}/*train-android-aarch64* ./train
-    file_name=$(ls ${arm64_path}/*inference-android-aarch64.tar.gz)
+    file_name=$(ls ${arm64_path}/*android-aarch64.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
@@ -1650,7 +1650,7 @@ if [[ $backend == "all" || $backend == "arm64_cpu" || $backend == "arm64_fp16" ]
     # Run on arm64-fp16
     arm64_path=${release_path}/android_aarch64
     # mv ${arm64_path}/*train-android-aarch64* ./train
-    file_name=$(ls ${arm64_path}/*inference-android-aarch64.tar.gz)
+    file_name=$(ls ${arm64_path}/*android-aarch64.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
diff --git a/mindspore/lite/test/st/scripts/run_benchmark_gpu.sh b/mindspore/lite/test/st/scripts/run_benchmark_gpu.sh
index a0edf6c..cea1b1d 100644
--- a/mindspore/lite/test/st/scripts/run_benchmark_gpu.sh
+++ b/mindspore/lite/test/st/scripts/run_benchmark_gpu.sh
@@ -4,8 +4,8 @@
 function Run_Converter() {
     # Unzip x86 runtime and converter
     cd ${x86_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-linux-x64.tar.gz || exit 1
-    cd ${x86_path}/mindspore-lite-${version}-inference-linux-x64/ || exit 1
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    cd ${x86_path}/mindspore-lite-${version}-linux-x64/ || exit 1
 
     cp tools/converter/converter/converter_lite ./ || exit 1
     export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:./tools/converter/lib/:./tools/converter/third_party/glog/lib
@@ -114,19 +114,19 @@ function Run_Converter() {
 # Run on gpu platform:
 function Run_gpu() {
     cd ${arm64_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch64.tar.gz || exit 1
+    tar -zxf mindspore-lite-${version}-android-aarch64.tar.gz || exit 1
 
     # If build with minddata, copy the minddata related libs
     cd ${benchmark_test_path} || exit 1
-    if [ -f ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libminddata-lite.so ]; then
-        cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+    if [ -f ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ]; then
+        cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
     fi
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
 
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -326,7 +326,7 @@ done
 # mkdir train
 
 x86_path=${release_path}/ubuntu_x86
-file_name=$(ls ${x86_path}/*inference-linux-x64.tar.gz)
+file_name=$(ls ${x86_path}/*linux-x64.tar.gz)
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version=${file_name_array[2]}
 
@@ -387,7 +387,7 @@ if [[ $backend == "all" || $backend == "gpu" ]]; then
     # Run on gpu
     arm64_path=${release_path}/android_aarch64
     # mv ${arm64_path}/*train-android-aarch64* ./train
-    file_name=$(ls ${arm64_path}/*inference-android-aarch64.tar.gz)
+    file_name=$(ls ${arm64_path}/*android-aarch64.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
diff --git a/mindspore/lite/test/st/scripts/run_benchmark_npu.sh b/mindspore/lite/test/st/scripts/run_benchmark_npu.sh
index 892be5a..ca9e2a5 100644
--- a/mindspore/lite/test/st/scripts/run_benchmark_npu.sh
+++ b/mindspore/lite/test/st/scripts/run_benchmark_npu.sh
@@ -4,8 +4,8 @@
 function Run_Converter() {
     # Unzip x86 runtime and converter
     cd ${x86_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-linux-x64.tar.gz || exit 1
-    cd ${x86_path}/mindspore-lite-${version}-inference-linux-x64/ || exit 1
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    cd ${x86_path}/mindspore-lite-${version}-linux-x64/ || exit 1
 
     cp tools/converter/converter/converter_lite ./ || exit 1
     export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:./tools/converter/lib/:./tools/converter/third_party/glog/lib
@@ -59,19 +59,19 @@ function Run_Converter() {
 # Run on npu platform:
 function Run_npu() {
     cd ${arm64_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch64.tar.gz || exit 1
+    tar -zxf mindspore-lite-${version}-android-aarch64.tar.gz || exit 1
 
     # If build with minddata, copy the minddata related libs
     cd ${benchmark_test_path} || exit 1
-    if [ -f ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/minddata/lib/libminddata-lite.so ]; then
-        cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/minddata/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
+    if [ -f ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/minddata/lib/libminddata-lite.so ]; then
+        cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/minddata/lib/libminddata-lite.so ${benchmark_test_path}/libminddata-lite.so || exit 1
     fi
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai.so ${benchmark_test_path}/libhiai.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir.so ${benchmark_test_path}/libhiai_ir.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so ${benchmark_test_path}/libhiai_ir_build.so || exit 1
 
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/inference/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
-    cp -a ${arm64_path}/mindspore-lite-${version}-inference-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/runtime/lib/libmindspore-lite.so ${benchmark_test_path}/libmindspore-lite.so || exit 1
+    cp -a ${arm64_path}/mindspore-lite-${version}-android-aarch64/tools/benchmark/benchmark ${benchmark_test_path}/benchmark || exit 1
 
     # adb push all needed files to the phone
     adb -s ${device_id} push ${benchmark_test_path} /data/local/tmp/ > adb_push_log.txt
@@ -178,7 +178,7 @@ done
 # mkdir train
 
 x86_path=${release_path}/ubuntu_x86
-file_name=$(ls ${x86_path}/*inference-linux-x64.tar.gz)
+file_name=$(ls ${x86_path}/*linux-x64.tar.gz)
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version=${file_name_array[2]}
 
@@ -235,7 +235,7 @@ if [[ $backend == "all" || $backend == "npu" ]]; then
     # Run on npu
     arm64_path=${release_path}/android_aarch64
     # mv ${arm64_path}/*train-android-aarch64* ./train
-    file_name=$(ls ${arm64_path}/*inference-android-aarch64.tar.gz)
+    file_name=$(ls ${arm64_path}/*android-aarch64.tar.gz)
     IFS="-" read -r -a file_name_array <<< "$file_name"
     version=${file_name_array[2]}
 
diff --git a/mindspore/lite/test/st/scripts/run_benchmark_x86.sh b/mindspore/lite/test/st/scripts/run_benchmark_x86.sh
index 23ec599..1328939 100644
--- a/mindspore/lite/test/st/scripts/run_benchmark_x86.sh
+++ b/mindspore/lite/test/st/scripts/run_benchmark_x86.sh
@@ -4,8 +4,8 @@
 function Run_Converter() {
     # Unzip x86 runtime and converter
     cd ${x86_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-linux-x64.tar.gz || exit 1
-    cd ${x86_path}/mindspore-lite-${version}-inference-linux-x64/ || exit 1
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    cd ${x86_path}/mindspore-lite-${version}-linux-x64/ || exit 1
 
     cp tools/converter/converter/converter_lite ./ || exit 1
     export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:./tools/converter/lib/:./tools/converter/third_party/glog/lib
@@ -344,9 +344,9 @@ function Run_Converter() {
 
 # Run on x86 platform:
 function Run_x86() {
-    echo 'cd  '${x86_path}'/mindspore-lite-'${version}'-inference-linux-x64' >> "${run_x86_log_file}"
-    cd ${x86_path}/mindspore-lite-${version}-inference-linux-x64 || return 1
-    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./inference/lib
+    echo 'cd  '${x86_path}'/mindspore-lite-'${version}'-linux-x64' >> "${run_x86_log_file}"
+    cd ${x86_path}/mindspore-lite-${version}-linux-x64 || return 1
+    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./runtime/lib
     cp tools/benchmark/benchmark ./ || exit 1
 
     # Run tf converted models:
@@ -671,9 +671,9 @@ function Run_x86() {
 # Run on x86 sse platform:
 function Run_x86_sse() {
     cd ${x86_path}/sse || exit 1
-    tar -zxf mindspore-lite-${version}-inference-linux-x64.tar.gz || exit 1
-    cd ${x86_path}/sse/mindspore-lite-${version}-inference-linux-x64 || return 1
-    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./inference/lib
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    cd ${x86_path}/sse/mindspore-lite-${version}-linux-x64 || return 1
+    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./runtime/lib
     cp tools/benchmark/benchmark ./ || exit 1
 
     # Run tflite converted models:
@@ -943,9 +943,9 @@ function Run_x86_sse() {
 # Run on x86 avx platform:
 function Run_x86_avx() {
     cd ${x86_path}/avx || exit 1
-    tar -zxf mindspore-lite-${version}-inference-linux-x64.tar.gz || exit 1
-    cd ${x86_path}/avx/mindspore-lite-${version}-inference-linux-x64 || return 1
-    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./inference/lib
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    cd ${x86_path}/avx/mindspore-lite-${version}-linux-x64 || return 1
+    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./runtime/lib
     cp tools/benchmark/benchmark ./ || exit 1
 
     # Run tflite converted models:
@@ -1215,11 +1215,11 @@ function Run_x86_avx() {
 # Run on x86 java platform:
 function Run_x86_java() {
     cd ${x86_java_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-linux-x64.tar.gz || exit 1
-    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:${x86_java_path}/mindspore-lite-${version}-inference-linux-x64/inference/lib
+    tar -zxf mindspore-lite-${version}-linux-x64.tar.gz || exit 1
+    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:${x86_java_path}/mindspore-lite-${version}-linux-x64/runtime/lib
     # compile benchmark
-    echo "javac -cp ${x86_java_path}/mindspore-lite-${version}-inference-linux-x64/inference/lib/mindspore-lite-java.jar ${basepath}/java/src/main/java/Benchmark.java -d ."
-    javac -cp ${x86_java_path}/mindspore-lite-${version}-inference-linux-x64/inference/lib/mindspore-lite-java.jar ${basepath}/java/src/main/java/Benchmark.java -d .
+    echo "javac -cp ${x86_java_path}/mindspore-lite-${version}-linux-x64/runtime/lib/mindspore-lite-java.jar ${basepath}/java/src/main/java/Benchmark.java -d ."
+    javac -cp ${x86_java_path}/mindspore-lite-${version}-linux-x64/runtime/lib/mindspore-lite-java.jar ${basepath}/java/src/main/java/Benchmark.java -d .
 
     count=0
     # Run tflite converted models:
@@ -1234,8 +1234,8 @@ function Run_x86_java() {
           continue
         fi
         echo ${model_name} >> "${run_x86_java_log_file}"
-        echo "java -classpath .:${x86_java_path}/mindspore-lite-${version}-inference-linux-x64/inference/lib/mindspore-lite-java.jar Benchmark ${ms_models_path}/${model_name}.ms '${models_path}'/input_output/input/${model_name}.ms.bin '${models_path}'/input_output/output/${model_name}.ms.out 1" >> "${run_x86_java_log_file}"
-        java -classpath .:${x86_java_path}/mindspore-lite-${version}-inference-linux-x64/inference/lib/mindspore-lite-java.jar Benchmark ${ms_models_path}/${model_name}.ms ${models_path}/input_output/input/${model_name}.ms.bin ${models_path}/input_output/output/${model_name}.ms.out 1
+        echo "java -classpath .:${x86_java_path}/mindspore-lite-${version}-linux-x64/runtime/lib/mindspore-lite-java.jar Benchmark ${ms_models_path}/${model_name}.ms '${models_path}'/input_output/input/${model_name}.ms.bin '${models_path}'/input_output/output/${model_name}.ms.out 1" >> "${run_x86_java_log_file}"
+        java -classpath .:${x86_java_path}/mindspore-lite-${version}-linux-x64/runtime/lib/mindspore-lite-java.jar Benchmark ${ms_models_path}/${model_name}.ms ${models_path}/input_output/input/${model_name}.ms.bin ${models_path}/input_output/output/${model_name}.ms.out 1
         if [ $? = 0 ]; then
             run_result='x86_java: '${model_name}' pass'; echo ${run_result} >> ${run_benchmark_result_file}
         else
@@ -1246,7 +1246,7 @@ function Run_x86_java() {
 
 # Run on x86 codegen benchmark
 function Run_x86_codegen() {
-    local CODEGEN_PATH=${x86_path}/mindspore-lite-${version}-inference-linux-x64/tools/codegen
+    local CODEGEN_PATH=${x86_path}/mindspore-lite-${version}-linux-x64/tools/codegen
 
     rm -rf ${build_path}
     mkdir -p ${build_path}
@@ -1261,7 +1261,7 @@ function Run_x86_codegen() {
         ${CODEGEN_PATH}/codegen --codePath=${build_path} --modelPath=${ms_models_path}/${model_name}.ms --supportParallel=${PARALLEL} >> ${run_x86_codegen_log_file}
         # 1. build benchmark
         mkdir -p ${build_path}/${model_name}/build && cd ${build_path}/${model_name}/build || exit 1
-        cmake -DPKG_PATH=${x86_path}/mindspore-lite-${version}-inference-linux-x64 ${build_path}/${model_name} >> ${run_x86_codegen_log_file}
+        cmake -DPKG_PATH=${x86_path}/mindspore-lite-${version}-linux-x64 ${build_path}/${model_name} >> ${run_x86_codegen_log_file}
         make >> ${run_x86_codegen_log_file}
         # 2. run benchmark
         echo "net file: ${build_path}/${model_name}/src/net.bin" >> ${run_x86_codegen_log_file}
@@ -1341,7 +1341,7 @@ done
 # mkdir train
 
 x86_path=${release_path}/ubuntu_x86
-file_name=$(ls ${x86_path}/*inference-linux-x64.tar.gz)
+file_name=$(ls ${x86_path}/*linux-x64.tar.gz)
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version=${file_name_array[2]}
 
diff --git a/mindspore/lite/test/st/scripts/run_cropper.sh b/mindspore/lite/test/st/scripts/run_cropper.sh
index 5aef974..13b013f 100644
--- a/mindspore/lite/test/st/scripts/run_cropper.sh
+++ b/mindspore/lite/test/st/scripts/run_cropper.sh
@@ -24,16 +24,16 @@ function Print_Cropper_Result() {
 
 function Run_cropper() {
     cd ${arm64_path} || exit 1
-    tar -zxf mindspore-lite-${version}-inference-android-aarch64.tar.gz || exit 1
-    cd mindspore-lite-${version}-inference-android-aarch64 || exit 1
-    cp -a ./inference/third_party/hiai_ddk/lib/libhiai.so "${cropper_test_path}"/libhiai.so || exit 1
-    cp -a ./inference/third_party/hiai_ddk/lib/libhiai_ir.so "${cropper_test_path}"/libhiai_ir.so || exit 1
-    cp -a ./inference/third_party/hiai_ddk/lib/libhiai_ir_build.so "${cropper_test_path}"/libhiai_ir_build.so || exit 1
+    tar -zxf mindspore-lite-${version}-android-aarch64.tar.gz || exit 1
+    cd mindspore-lite-${version}-android-aarch64 || exit 1
+    cp -a ./runtime/third_party/hiai_ddk/lib/libhiai.so "${cropper_test_path}"/libhiai.so || exit 1
+    cp -a ./runtime/third_party/hiai_ddk/lib/libhiai_ir.so "${cropper_test_path}"/libhiai_ir.so || exit 1
+    cp -a ./runtime/third_party/hiai_ddk/lib/libhiai_ir_build.so "${cropper_test_path}"/libhiai_ir_build.so || exit 1
 
-    cp -a ./inference/lib/libmindspore-lite.a "${cropper_test_path}"/libmindspore-lite.a || exit 1
+    cp -a ./runtime/lib/libmindspore-lite.a "${cropper_test_path}"/libmindspore-lite.a || exit 1
     cp -a ./tools/benchmark/benchmark "${cropper_test_path}"/benchmark || exit 1
 
-    cp -r "${x86_path}"/mindspore-lite-${version}-inference-linux-x64/tools/cropper/ "${cropper_test_path}" || exit 1
+    cp -r "${x86_path}"/mindspore-lite-${version}-linux-x64/tools/cropper/ "${cropper_test_path}" || exit 1
 
     cd "${cropper_test_path}" || exit 1
     echo "${cropper_test_path}"
@@ -137,7 +137,7 @@ echo ' ' > "${run_converter_log_file}"
 run_converter_result_file="${basepath}"/run_converter_result.txt
 echo ' ' > "${run_converter_result_file}"
 
-file_name=$(ls "${x86_path}"/*inference-linux-x64.tar.gz)
+file_name=$(ls "${x86_path}"/*linux-x64.tar.gz)
 IFS="-" read -r -a file_name_array <<< "$file_name"
 version=${file_name_array[2]}
 ms_models_path=${basepath}/ms_models
